In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import pickle
from math import floor
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dense, Dropout, Flatten, MaxPooling2D, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import History 
from tensorflow.keras import Model
from tensorflow.keras import backend as K
if K.backend() == 'tensorflow':
    import tensorflow as tf

In [3]:
train_path = './seg_train/seg_train/'
test_path = './seg_test/seg_test/'
Labels = ['buildings', 'forest','glacier','mountain','sea','street']
label_dict = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}

In [4]:
min_count = len(os.listdir(train_path+'buildings'))*3

In [5]:
def get_data(path):
    #train_label_count = {'buildings':0, 'forest':0, 'glacier':0, 'mountain':0, 'sea':0, 'street':0}
    images = list()
    lbls = list()
    for label in Labels:
        count = 0
        for image in os.listdir(path+label):
            if count < min_count:
                img = cv2.imread(path+label+r'/'+image)
                img = Image.fromarray(img , 'RGB')
                img = img.resize((50,50))
                images.append(np.array(img)/255)
                if label == 'buildings':
                    lbl = 0
                elif label == 'forest':
                    lbl = 1
                elif label == 'glacier':
                    lbl = 2
                elif label == 'mountain':
                    lbl = 3
                elif label == 'sea':
                    lbl = 4
                else:
                    lbl = 5
                lbls.append(lbl)
                if path == train_path:
                    rotated = img.rotate(45)
                    flipped = np.fliplr(img)
                    images.append(np.array(rotated)/255)
                    images.append(np.array(flipped)/255)
                    lbls.append(lbl)
                    lbls.append(lbl)
                    #train_label_count[label] += 3
                    count += 3
                
    images = np.array(images)
    lbls = np.array(lbls)
    
    return shuffle(images, lbls, random_state=42)

In [6]:
train_images, train_labels = get_data(train_path)

In [7]:
test_images, test_labels = get_data(test_path)

In [8]:
#samples_len = floor(len(train_images)/6)

In [9]:
monitor = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [20]:
train_A, train_B = train_images, train_images

In [21]:
input_A = Input(shape=train_A.shape[1:], name='input_A')
flat_A = Flatten()(input_A)
input_B = Input(shape=train_B.shape[1:], name='input_B')
conv_1 = Conv2D(filters=32,kernel_size=2,padding="same",activation="relu")(input_B)
pool_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
conv_2 = Conv2D(filters=32,kernel_size=2,padding="same",activation="relu")(pool_1)
pool_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
drop_1 = Dropout(0.2)(pool_2)
flat_2 = Flatten()(drop_1)
hidden1 = Dense(500, activation='relu')(flat_2)
drop_2 = Dropout(0.2)(hidden1)
concat = Concatenate()([flat_A, drop_2])
output = Dense(6, activation="softmax")(concat)
model = Model(inputs=[input_A, input_B], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_B (InputLayer)            [(None, 50, 50, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 50, 50, 32)   416         input_B[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 25, 25, 32)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 25, 25, 32)   4128        max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [24]:
model.fit([train_A, train_B], train_labels, epochs=50,validation_split=.3, callbacks=[monitor])

Train on 27606 samples, validate on 11832 samples
Epoch 1/50
27606/27606 [==============================] - 41s 1ms/sample - loss: 1.2241 - accuracy: 0.5507 - val_loss: 0.8911 - val_accuracy: 0.6611
Epoch 2/50
27606/27606 [==============================] - 41s 2ms/sample - loss: 0.9554 - accuracy: 0.6603 - val_loss: 0.8114 - val_accuracy: 0.7093
Epoch 3/50
27606/27606 [==============================] - 41s 1ms/sample - loss: 0.8859 - accuracy: 0.6984 - val_loss: 0.7958 - val_accuracy: 0.7170
Epoch 4/50
27606/27606 [==============================] - 41s 1ms/sample - loss: 0.7599 - accuracy: 0.7347 - val_loss: 0.7820 - val_accuracy: 0.7221
Epoch 5/50
27606/27606 [==============================] - 40s 1ms/sample - loss: 0.7058 - accuracy: 0.7567 - val_loss: 0.7431 - val_accuracy: 0.7546
Epoch 6/50
27606/27606 [==============================] - 41s 1ms/sample - loss: 0.6275 - accuracy: 0.7825 - val_loss: 0.7663 - val_accuracy: 0.7675
Epoch 7/50
27606/27606 [==============================] 

In [25]:
def build_ensemble():
    models = list()
    for i in range(1,7):
        model=Sequential()
        model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=train_images.shape[1:]))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(250*i,activation="relu"))
        model.add(Dropout(0.2))
        model.add(Dense(6,activation="softmax"))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.summary()
        models.append(model)
    return models

In [26]:
def train_ensemble(train_images, train_labels):
    #new_train_images, new_train_labels = train_images, train_labels
    trained_models = list()
    models = build_ensemble()
    for model in models:
        #model_train_images = new_train_images[:samples_len]
        #model_train_labels = new_train_labels[:samples_len]
        model.fit(train_images,train_labels,epochs=50,validation_split=.3, callbacks=[monitor])
        trained_models.append(model)
        #new_train_images = new_train_images[samples_len:]
        #new_train_labels = new_train_labels[samples_len:]
        
    return trained_models

In [27]:
#trained_ensemble = train_ensemble(train_images, train_labels)

In [28]:
#for model in trained_ensemble:
#    model.evaluate(test_images,test_labels, verbose=1)

In [30]:
model.evaluate([test_images,test_images],test_labels, verbose=1)

3000/3000 [==============================] - 1s 395us/sample - loss: 0.7632 - accuracy: 0.7470


[0.7632287295659383, 0.747]